# A Deep Hierarchical Variational Autoencoder （一种深层次变分自编码器）

## 绪论
在标准化流，自回归模型，变分自编码器，深度基于能量模型之间，角逐谁才是深度生成学习中最好的模型框架。在其中，VAEs具有快速，易处理采样和易于接入编码网络的优点。其表现优于其他模型。虽然VAEs的大部分研究都集中在统计挑战上，我们探索了为分层VAE仔细设计神经结构的正交方向。利用**深度可分离卷积**和**批量归一化**方法，构建了用于图像生成的深度层次VAE。NVAE具有正态分布的残差参数化，其训练通过谱正则化来稳定。我们在MNIST、CIFAR - 10、Celeb A 64和Celeb A HQ数据集上展示了NVAE在非自回归似然模型中取得的最新结果，并在FFHQ上提供了一个强有力的基线。NVAE是第一个成功用于生成像素$256\times 256$自然图像的VAE。此处浏览[源代码](https://github.com/NVlabs/NVAE)

## 序言
关于提升VAEs性能的研究大多都集中于统计挑战上。例如缩小近似和真实后验分布之间的差距，制定更严格的界限，降低梯度噪声，将VAEs扩展到离散变量，或处理后验衰弱。对于VAEs，**神经网络结构的作用有些被忽略**，因为大多数以前的工作都是从分类任务中借用结构。然而，VAEs可以受益于设计特殊的网络体系结构，因为它们有着根本不同的要求。首先，VAEs最大化了输入变量和潜变量之间的互信息，要求网络尽可能保留输入数据的信息内容。这与丢弃输入信息的分类网络相反。第二，VAEs对神经网络中的过参数化反应往往不同。由于边际对数似然只取决于生成模型，解码器网络的过度参数化可能会损害测试日志似然，而强大的编码器可以产生更好的模型，因为减少了摊销差距。请注意，由非基于编码器的方法估计的边际对数似然对编码器过拟合不敏感。此外，用于VAE的神经网络应该对数据中的长程相关性进行建模，这要求网络具有较大的感受域。最后，由于变分下界中的无界( KL )散度，训练非常深的分层VAE通常是不稳定的。当前最先进的VAEs忽略了批标准化( BN )，以对抗可能会放大其不稳定性的随机性来源。

在本文中，我们旨在通过架构设计使VAEs再次伟大。我们提出了新的VAE ( NVAE )，它是一个深度层次的VAE，具有精心设计的网络体系结构，产生高质量的图像。NVAE在基于非自回归似然的生成模型中得到了最新的结果，缩小了与自回归模型的差距。我们网络的主要组成部分是深度卷积，它在不明显增加参数数目的情况下迅速增加网络的接收场。

与前面的工作相反，我们发现BN是深度VAE成功的重要组成部分。我们还注意到，当等级组的数量增加时，培训的不稳定性仍然是一个主要障碍，与BN的存在无关。为了解决这个问题，我们提出了近似后验参数的残差参数化来改进最小化KL项，并且我们表明谱正则化是稳定VAE训练的关键。

总之，我们做出了以下贡献：( 1 )提出了一种新的深度层次VAE，称为NVAE，其生成模型中包含了深度卷积；( 2 )提出了一种新的近似后验残差参数化方法；( 3 )利用谱正则化方法稳定训练深度VAEs；( 4 )为减少VAEs的内存负担提供了切实可行的解决方案；( 5 )我们证明了深度层次VAEs能够在多个图像数据集上获得最新的结果，即使在原始VAE目标下训练，也能产生高质量的样本。据我们所知，NVAE是VAEs首次成功地应用于高达256256像素的图像。

相关工作：最近，VQ-VAE-2 对大图像显示了高质量的生成性能。尽管VQ - VAE的提出是由VAEs驱动的，但其目标并不对应于数据对数似然度的下界。相比之下，NVAE是通过VAE目标直接训练的。此外，VQ-VAE-2在其先前使用Pixel CNN对128128维的潜变量进行采样非常缓慢，而NVAE在数据空间使用无条件解码器。

我们的工作与具有逆自回归流的VAEs ( IAF-VAEs )有关。NVAE从IAF-VAEs中借用了统计模型( 即层次先验和近似后验等 )。但与IAF-VAEs不同的是，i )实现这些模型的神经网络，ii )近似后验参数化，以及iii )扩大训练到大图像。尽管如此，我们提供了这些方面的烧蚀实验，我们表明NVAE比原IAF-VAEs有很大的差距。最近，BIVA 将双向推理扩展到潜变量，显示了最新的VAE结果。然而，BIVA使用了类似于IAF-VAE的神经网络，它在高达6464px的图像上进行训练。为了保持事情简单，我们采用了来自IAF-VAEs的递阶层次结构，并着重对神经网络进行了精心设计。如果使用BIVA中更为复杂的层次结构模型，我们期望性能得到改善。早期的工作DRAW 和Conv DRAW 使用递归神经网络来建模层次依赖关系。

## 背景
